# Test Reading Metadata from BWF Wave file

In [54]:
from bwfsoundfile import BwfSoundFile
import librosa
from IPython.display import display, Audio

In [55]:
file = BwfSoundFile('./Lethal_Energies_orig/Designed Weapons/DS_Heavy/DS_Heavy_01_Shot-1.wav')

In [56]:
file.read_metadata()

In [57]:
print (file.ixml_info)

<?xml version="1.0" encoding="UTF-8"?><BWFXML><IXML_VERSION>1.52</IXML_VERSION><SCENE></SCENE><TAKE></TAKE><TAPE></TAPE><PROJECT></PROJECT><FILE_UID></FILE_UID></BWFXML>


In [58]:
print (file.bext_info)

{'version': 0, 'originator_reference': '', 'originator': '', 'description': 'sci-fi, futuristic, weapon, gun, cannon, launcher, blast, heavy, huge, massive', 'origination_time': '', 'umid': '', 'coding_history': '', 'timereference_translated': '00:00:00.000000', 'timereference': 0, 'origination_date': '2015'}


In [59]:
print (file.get_core_info())

{'bit_depth_info': 'Signed 24 bit PCM', 'bit_depth': 24, 'channels': 2, 'samplerate': 48000, 'duration': '00:00:02'}


In [60]:
samplerate = file.samplerate
print (samplerate)

48000


# Test Downsample to 16KHz

In [61]:
data = file.read(-1, dtype='float32')
data = data.T

In [62]:
display(Audio(data, rate=samplerate))

In [63]:
resamplerate = 16000
data_resampled = librosa.resample(data, samplerate, resamplerate)
display(Audio(data_resampled, rate=resamplerate))

# Test Writing to file

In [67]:
import soundfile as sf
import os

In [74]:
out_dir = 'Lethal_Energies_Preprocessed/'
filename = file.name.split('/')[-1]
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
sf.write(out_dir + filename, data_resampled.T, resamplerate, subtype='PCM_16')

In [75]:
file.close()

# Test Recursive Search for Wave Files

In [81]:
import glob
import numpy as np

In [92]:
file_list = []
for filename in glob.iglob('Lethal_Energies_orig/**/*.wav', recursive=True):
    file_list.append(filename)
file_array = np.array(file_list)
print (file_array)
print (file_array.size)

['Lethal_Energies_orig\\Construction Kit\\CK_Aiming\\CK_Aiming-01.wav'
 'Lethal_Energies_orig\\Construction Kit\\CK_Aiming\\CK_Aiming-02.wav'
 'Lethal_Energies_orig\\Construction Kit\\CK_Aiming\\CK_Aiming-03.wav' ...
 'Lethal_Energies_orig\\Designed Weapons\\DS_Medium\\DS_Medium_10_Shot-4.wav'
 'Lethal_Energies_orig\\Designed Weapons\\DS_Medium\\DS_Medium_10_Shot-5.wav'
 'Lethal_Energies_orig\\Designed Weapons\\DS_Medium\\DS_Medium_10_Shot-6.wav']
1079


# Bulk Resample

In [14]:
from bwfsoundfile import BwfSoundFile
import soundfile as sf
import librosa
from IPython.display import display, Audio
import glob
import os

In [16]:
resamplerate = 16000
out_dir = 'Swordfighter_Preprocessed/'
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

In [17]:
file_list = []
for filename in glob.iglob('Swordfighter_orig/Swordfighter - Video Games/wav/**/*.wav', recursive=True):
    file_list.append(filename)

In [18]:
for filename in file_list:
    with BwfSoundFile(filename) as bwf_file:
        # Read
        samplerate = bwf_file.samplerate
        data = bwf_file.read(-1, dtype='float32')
        data = data.T # Librosa uses shape (nb_channels, nb_samples), PySoundFile uses shape (nb_samples, nb_channels)
        
        # Resample
        data_resampled = librosa.resample(data, samplerate, resamplerate)
        
        # Write
        out_filename = bwf_file.name.split('\\')[-1]
        sf.write(out_dir + out_filename, data_resampled.T, resamplerate, subtype='PCM_16')